---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [58]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [59]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [62]:
def graph_identification():
    
    # Your Code Here1 - PA, 4 - PA, 2,3-SW_L, 5 - SW_H
    return ['PA','SW_L','SW_L','PA','SW_H']
    #return nx.average_clustering(P1_Graphs[0]), nx.average_clustering(P1_Graphs[1]), nx.average_clustering(P1_Graphs[2]), nx.average_clustering(P1_Graphs[3]), nx.average_clustering(P1_Graphs[4])# Your Answer Here
graph_identification()
#(4.099161161161161,5.089871871871872, 9.378702269692925,3.1048046283934134,5.0785509568313305)__avg
#(0.03167539146454044,0.5642419635919628,0.4018222222222227,0.03780379975223251,0.0033037037037037037)__clus

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [63]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [64]:
def salary_predictions():
    
    # Your Code Here
    count_unpaid = []
    #Unpaid = [x[0] for x in G.nodes(data = True) if np.isnan(x[1]['ManagementSalary'])]
    df = pd.DataFrame(G.nodes(data = True), columns = ['Node', 'data'])
    df['Department'] = df['data'].map(lambda x:x['Department'])
    df['ManagementSalary'] = df['data'].map(lambda x:x['ManagementSalary'])
    df = df[['Node','Department', 'ManagementSalary']]
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pagerank'] = pd.Series(nx.pagerank(G))
    df['hub_score'] = pd.Series(nx.hits(G, normalized=True)[0])
    df['auth_score'] = pd.Series(nx.hits(G, normalized=True)[1])
    nans = df[df.isnull().any(axis=1)]
    nans_not = df[df.isnull().any(axis=1) == False]
    #return len(nans), len(nans_not), len(nans)+len(nans_not), len(df) (252, 753, 1005, 1005)
    #return nans['ManagementSalary'].unique(), nans_not['ManagementSalary'].unique()   (array([ nan]), array([ 0.,  1.]))
    X_train = nans_not[['Department','clustering','degree','degree_centrality','closeness','betweeness','pagerank', 'hub_score', 'auth_score']]
    y_train = nans_not['ManagementSalary']
    X_test  = nans[['Department','clustering','degree','degree_centrality','closeness','betweeness','pagerank', 'hub_score', 'auth_score']]
    #return X_test
    #return len(X_train), len(y_train), len(X_test)
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import roc_auc_score
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LogisticRegression
    model = LogisticRegression()
    scalar = MinMaxScaler()
    X_train_scaled = scalar.fit_transform(X_train)
    X_test_scaled = scalar.transform(X_test)
    grid_clf_auc = GridSearchCV(model, param_grid = {'C': [0.01,0.1,5,20,50,200]} , scoring = 'roc_auc')
    grid_clf_auc.fit(X_train_scaled, y_train)
    #return X_train_scaled
    #grid_clf_auc.probability = True
    return pd.Series(grid_clf_auc.predict_proba(X_test_scaled)[:,1], X_test.index)# Your Answer Here
salary_predictions()

1       3.906539e-02
2       8.001967e-01
5       9.999879e-01
8       1.229470e-01
14      1.681388e-01
18      2.585436e-01
27      2.643140e-01
30      2.990336e-01
31      6.922048e-02
34      8.015665e-02
37      2.266849e-02
40      6.202600e-02
45      1.255557e-01
54      2.272876e-01
55      5.622735e-01
60      4.512333e-02
62      9.996682e-01
65      9.555907e-01
77      4.170280e-02
79      9.446087e-02
97      5.956495e-02
101     1.595535e-02
103     2.282789e-01
108     9.001598e-02
113     4.242115e-01
122     3.909752e-03
141     4.761482e-01
142     8.195258e-01
144     6.116880e-02
145     6.546273e-01
            ...     
913     6.918162e-03
914     2.383522e-02
915     9.264846e-07
918     1.136451e-01
923     1.438328e-03
926     5.197741e-02
931     1.553892e-02
934     5.746381e-04
939     1.581122e-04
944     1.958405e-06
945     3.603949e-03
947     1.556669e-01
950     7.734224e-02
951     2.594086e-03
953     2.309986e-03
959     5.309826e-06
962     1.629

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [65]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)
#488446

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [66]:
def new_connections_predictions():
    
    # Your Code Here
    df = future_connections
    df.columns = ['connections']
    pref_att = sorted(list(nx.preferential_attachment(G)), key = lambda x:[x[0],x[1]])
    df_g = pd.DataFrame(index = [(edge[0],edge[1]) for edge in pref_att])
    df_g['pref_att'] = [x[2] for x in pref_att]
    L_jc = sorted(list(nx.jaccard_coefficient(G)), key = lambda x:[x[0],x[1]])
    df_g['jc'] = [x[2] for x in L_jc]
    L_ra =  sorted(list(nx.resource_allocation_index(G)), key = lambda x:[x[0],x[1]])
    df_g['ra'] = [x[2] for x in L_ra]
    L_ac = sorted(list(nx.adamic_adar_index(G)), key = lambda x:[x[0],x[1]])
    df_g['ac'] = [x[2] for x in L_ac]
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    L_sc = sorted(list(nx.cn_soundarajan_hopcroft(G)), key = lambda x:[x[0],x[1]])
    df_g['sc'] = [x[2] for x in L_ac]
    df_mer = df.join(df_g)
    df_mer['pref_att'].fillna(0, inplace = True)
    df_mer['jc'].fillna(0, inplace = True)
    df_mer['ra'].fillna(0, inplace = True)
    df_mer['ac'].fillna(0, inplace = True)
    df_mer['sc'].fillna(0, inplace = True)
    nans = df_mer[df_mer.isnull().any(axis=1)]
    nans_not = df_mer[df_mer.isnull().any(axis=1) == False]
    #return len(nans), len(nans_not), len(nans)+len(nans_not), len(df) (252, 753, 1005, 1005)
    #return nans['ManagementSalary'].unique(), nans_not['ManagementSalary'].unique()   (array([ nan]), array([ 0.,  1.]))
    X_train = nans_not[['pref_att','jc','ra','ac','sc']]
    y_train = nans_not['connections']
    X_test  = nans[['pref_att','jc','ra','ac','sc']]
    #return X_test
    #return len(X_train), len(y_train), len(X_test)
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import roc_auc_score
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LogisticRegression
    model = LogisticRegression()
    scalar = MinMaxScaler()
    X_train_scaled = scalar.fit_transform(X_train)
    X_test_scaled = scalar.transform(X_test)
    grid_clf_auc = GridSearchCV(model, param_grid = {'C': [0.01,0.1,5,20,50,200]} , scoring = 'roc_auc')
    grid_clf_auc.fit(X_train_scaled, y_train)
    #return X_train_scaled
    #grid_clf_auc.probability = True
    return pd.Series(grid_clf_auc.predict_proba(X_test_scaled)[:,1], X_test.index)# Your Answer Here
new_connections_predictions()

(107, 348)    0.028184
(542, 751)    0.011220
(20, 426)     0.535988
(50, 989)     0.011268
(942, 986)    0.011321
(324, 857)    0.011262
(13, 710)     0.199829
(19, 271)     0.129855
(319, 878)    0.011285
(659, 707)    0.011225
(49, 843)     0.011301
(208, 893)    0.011236
(377, 469)    0.009291
(405, 999)    0.019609
(129, 740)    0.016297
(292, 618)    0.030123
(239, 689)    0.011289
(359, 373)    0.010073
(53, 523)     0.039919
(276, 984)    0.011298
(202, 997)    0.011308
(604, 619)    0.191963
(270, 911)    0.011288
(261, 481)    0.073508
(200, 450)    0.898537
(213, 634)    0.011204
(644, 735)    0.113328
(346, 553)    0.011083
(521, 738)    0.010847
(422, 953)    0.020995
                ...   
(672, 848)    0.011288
(28, 127)     0.920402
(202, 661)    0.011062
(54, 195)     0.999990
(295, 864)    0.011254
(814, 936)    0.011212
(839, 874)    0.011321
(139, 843)    0.011258
(461, 544)    0.010683
(68, 487)     0.010654
(622, 932)    0.011242
(504, 936)    0.017954
(479, 528) 